# AlexNet iFood2019 - Analysis and Visualization

This notebook provides comprehensive analysis of trained models:
1. Load all results
2. Compare training curves
3. Analyze metrics
4. Generate visualizations
5. Create summary report

In [ ]:
# ============================================================
# Cell 1: Setup
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os
import sys
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use('seaborn-v0_8-whitegrid')

PROJECT_PATH = '/content/drive/MyDrive/AlexNet_iFood2019'
CHECKPOINT_DIR = os.path.join(PROJECT_PATH, 'checkpoints')
EVAL_DIR = os.path.join(PROJECT_PATH, 'evaluation_results')
ANALYSIS_DIR = os.path.join(PROJECT_PATH, 'analysis_results')

MODELS = ['alexnet_baseline', 'alexnet_mod1', 'alexnet_mod2', 'alexnet_combined']

print("✓ Setup complete")

In [ ]:
# ============================================================
# Cell 2: Load Training Histories
# ============================================================

histories = {}

for model in MODELS:
    history_file = os.path.join(CHECKPOINT_DIR, f'{model}_history.json')
    if os.path.exists(history_file):
        with open(history_file) as f:
            histories[model] = json.load(f)
        print(f"✓ Loaded: {model}")
    else:
        print(f"⚠️  Not found: {model}")

print(f"\nLoaded {len(histories)} training histories")

In [ ]:
# ============================================================
# Cell 3: Load Evaluation Metrics
# ============================================================

metrics = {}

for model in MODELS:
    metrics_file = os.path.join(EVAL_DIR, f'{model}_val_metrics.json')
    if os.path.exists(metrics_file):
        with open(metrics_file) as f:
            metrics[model] = json.load(f)
        print(f"✓ Loaded: {model}")
    else:
        print(f"⚠️  Not found: {model}")

print(f"\nLoaded {len(metrics)} evaluation results")

In [ ]:
# ============================================================
# Cell 4: Training Curves Comparison
# ============================================================

if histories:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    colors = plt.cm.tab10(np.linspace(0, 1, len(histories)))
    
    # Train Loss
    ax = axes[0, 0]
    for (model, hist), color in zip(histories.items(), colors):
        epochs = range(1, len(hist['train_loss']) + 1)
        ax.plot(epochs, hist['train_loss'], color=color, label=model, linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Training Loss')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Val Loss
    ax = axes[0, 1]
    for (model, hist), color in zip(histories.items(), colors):
        epochs = range(1, len(hist['val_loss']) + 1)
        ax.plot(epochs, hist['val_loss'], color=color, label=model, linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Loss')
    ax.set_title('Validation Loss')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Train Acc
    ax = axes[1, 0]
    for (model, hist), color in zip(histories.items(), colors):
        epochs = range(1, len(hist['train_acc']) + 1)
        ax.plot(epochs, hist['train_acc'], color=color, label=model, linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.set_title('Training Accuracy')
    ax.legend()
    ax.grid(alpha=0.3)
    
    # Val Acc
    ax = axes[1, 1]
    for (model, hist), color in zip(histories.items(), colors):
        epochs = range(1, len(hist['val_acc']) + 1)
        ax.plot(epochs, hist['val_acc'], color=color, label=model, linewidth=2)
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy')
    ax.set_title('Validation Accuracy')
    ax.legend()
    ax.grid(alpha=0.3)
    
    plt.suptitle('Training Curves Comparison', fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(ANALYSIS_DIR, 'training_curves_detailed.png'), dpi=200)
    plt.show()
else:
    print("No training histories available")

In [ ]:
# ============================================================
# Cell 5: Metrics Bar Chart
# ============================================================

if metrics:
    metric_names = ['accuracy', 'top5_accuracy', 'precision', 'recall', 'macro_f1']
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    x = np.arange(len(metrics))
    width = 0.15
    
    for i, metric in enumerate(metric_names):
        values = [metrics[m].get(metric, 0) for m in metrics]
        bars = ax.bar(x + i * width, values, width, label=metric.replace('_', ' ').title())
        
        # Add value labels
        for bar, val in zip(bars, values):
            ax.annotate(f'{val:.3f}',
                       xy=(bar.get_x() + bar.get_width() / 2, bar.get_height()),
                       xytext=(0, 3), textcoords='offset points',
                       ha='center', va='bottom', fontsize=8, rotation=45)
    
    ax.set_xlabel('Model')
    ax.set_ylabel('Score')
    ax.set_title('Model Performance Comparison')
    ax.set_xticks(x + width * 2)
    ax.set_xticklabels([m.replace('alexnet_', '') for m in metrics])
    ax.legend(loc='upper right')
    ax.set_ylim(0, 1.15)
    ax.grid(alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.savefig(os.path.join(ANALYSIS_DIR, 'metrics_comparison_detailed.png'), dpi=200)
    plt.show()
else:
    print("No metrics available")

In [ ]:
# ============================================================
# Cell 6: Summary Table
# ============================================================

data = []

for model in MODELS:
    row = {'Model': model.replace('alexnet_', '').upper()}
    
    if model in histories:
        hist = histories[model]
        row['Train Loss (final)'] = f"{hist['train_loss'][-1]:.4f}"
        row['Best Val Acc'] = f"{max(hist['val_acc']):.4f}"
        row['Best Epoch'] = hist['val_acc'].index(max(hist['val_acc'])) + 1
    
    if model in metrics:
        m = metrics[model]
        row['Test Accuracy'] = f"{m.get('accuracy', 0):.4f}"
        row['Top-5 Accuracy'] = f"{m.get('top5_accuracy', 0):.4f}"
        row['Macro F1'] = f"{m.get('macro_f1', 0):.4f}"
    
    data.append(row)

df = pd.DataFrame(data)
print("\n" + "=" * 90)
print("MODEL COMPARISON SUMMARY")
print("=" * 90)
print(df.to_string(index=False))
print("=" * 90)

# Save to CSV
df.to_csv(os.path.join(ANALYSIS_DIR, 'model_comparison.csv'), index=False)
print(f"\n✓ Table saved to: {ANALYSIS_DIR}/model_comparison.csv")

In [ ]:
# ============================================================
# Cell 7: Best Model Analysis
# ============================================================

if metrics:
    # Find best model by accuracy
    best_model = max(metrics, key=lambda m: metrics[m].get('accuracy', 0))
    best_acc = metrics[best_model].get('accuracy', 0)
    
    print("🏆 BEST MODEL ANALYSIS")
    print("=" * 50)
    print(f"Model: {best_model}")
    print(f"Accuracy: {best_acc:.4f} ({best_acc*100:.2f}%)")
    print(f"Top-5 Accuracy: {metrics[best_model].get('top5_accuracy', 0):.4f}")
    print(f"Macro F1: {metrics[best_model].get('macro_f1', 0):.4f}")
    print(f"Weighted F1: {metrics[best_model].get('weighted_f1', 0):.4f}")
    print("=" * 50)
    
    # Improvement over baseline
    if 'alexnet_baseline' in metrics:
        baseline_acc = metrics['alexnet_baseline'].get('accuracy', 0)
        improvement = best_acc - baseline_acc
        print(f"\nImprovement over baseline: {improvement:.4f} ({improvement*100:.2f}%)")

In [ ]:
# ============================================================
# Cell 8: Generate Report Markdown
# ============================================================

report = f"""# AlexNet iFood2019 - Results Report

## Model Comparison

{df.to_markdown(index=False)}

## Key Findings

1. **Best Model**: {best_model}
2. **Best Accuracy**: {best_acc:.4f} ({best_acc*100:.2f}%)
3. **Top-5 Accuracy**: {metrics[best_model].get('top5_accuracy', 0):.4f}

## Training Observations

- All models trained for {NUM_EPOCHS if 'NUM_EPOCHS' in dir() else 50} epochs
- Batch size: 128
- Learning rate: 0.01 with step decay

## Plots

See generated images in `analysis_results/` folder.
"""

report_path = os.path.join(ANALYSIS_DIR, 'results_report.md')
with open(report_path, 'w') as f:
    f.write(report)

print(f"✓ Report saved to: {report_path}")
print("\n" + "=" * 50)
print("📊 ANALYSIS COMPLETE!")
print("=" * 50)